#### 这是改进后用于信息检索研究的代码，可以用于进行毕业设计的数据抓取（应当作为第一个执行的脚本）

In [1]:
import redis as redis
import requests
import random
import numpy as np
import selenium.webdriver as webdriver
from selenium.webdriver.common.by import By
import smallLib as helper
from time import time
import re
import os
#暴露了密码。但是这个密码是弱密码，不是重要的密码，所以就不改了
pool = redis.ConnectionPool(host='127.0.0.1',port=6379,db=0,password='123456',decode_responses=True)   
r = redis.Redis(connection_pool=pool)

In [3]:
#请在redistribution中创建一个InformationResearch的hash表，用来存储用户的信息
origin_table = "InformationResearch"
temp_table = "InformationResearch_temp"

#获取用户的信息
table =  r.hgetall(origin_table)

keys = r.keys()
if temp_table in keys:
    pass
else:
    r.hmset(temp_table,table)
user_videos ={}
#获取用户的视频信息
while r.hlen(temp_table)>0:
#由于抖音用户主页现在已经开始弹出登录框，所以这里使用selenium来获取用户主页的视频数
        try :
            user,url = helper.get_user(r,temp_table)
            #userHomepage = helper.getHtml(url).text.encode('utf-8')
           # video_num = int(re.findall("\"user-tab-count\">(.*?)</span>",userHomepage)[0])
            href ,video_num= helper.get_all_video_form_homepage(url)
            #douyin-right-container > div > div > div > div:nth-child(2) > div:nth-child(1) > div > div > div > div > h2 > span:nth-child(2)
            if video_num != 0 and 0.95*video_num <= len(href) :
                r.hdel(temp_table,user)
                r.hset("InformationResearch_videos",user,str(href))
            else:
                print(user)
            user_videos[user] = str(href)
            time.sleep(random.uniform(0.1,1.1))
        except Exception:
            continue

r.delete('curr',user)
cnt = 0
for key in user_videos.keys():
       cnt+= len(eval(user_videos[key]))
r.hmset("InformationResearch_videos",user_videos)



C:\Users\Dr.Seydlitz\AppData\Local\Temp\ipykernel_29364\300243390.py:12: DeprecationWarning: Redis.hmset() is deprecated. Use Redis.hset() instead.
  r.hmset(temp_table,table)


962
1140
272
316
妇科杨青梅教授
417
妇产科李主任
https://www.douyin.com/user/MS4wLjABAAAASgxFuOCGgcgubyGx7KQOfHUs0IQ67mdr4lc8v274jlF3mhoPtlWEBWK9EBFTq5Vg
👩🏻‍⚕️妇产科陈姐👩🏻‍⚕️
254
妇科郑晨思博士
606
妇产科梁培琴医生
171
妇产科舒大夫
1348
恩哥聊健康
79
妇科主任医师周坚红


Exception ignored in: <function Service.__del__ at 0x000001E9599C2480>
Traceback (most recent call last):
  File "d:\Anaconda\envs\pygptest\Lib\site-packages\selenium\webdriver\common\service.py", line 183, in __del__
    self.stop()
  File "d:\Anaconda\envs\pygptest\Lib\site-packages\selenium\webdriver\common\service.py", line 149, in stop
    self.send_remote_shutdown_command()
  File "d:\Anaconda\envs\pygptest\Lib\site-packages\selenium\webdriver\common\service.py", line 133, in send_remote_shutdown_command
    if not self.is_connectable():
           ^^^^^^^^^^^^^^^^^^^^^
  File "d:\Anaconda\envs\pygptest\Lib\site-packages\selenium\webdriver\common\service.py", line 122, in is_connectable
    return utils.is_connectable(self.port)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Anaconda\envs\pygptest\Lib\site-packages\selenium\webdriver\common\utils.py", line 102, in is_connectable
    socket_ = socket.create_connection((host, port), 1)
              ^^^^^^^^^^^^^^^^^^^^^^^^^

In [ ]:
origin_table = "InformationResearch_videos"
temp_download_table = "InformationResearch_download_temp"

table =  r.hgetall(origin_table)
r.hmset(temp_download_table,table)

pair=[1,1]
user= [1,1]
video_info = {};


C:\Users\Dr.Seydlitz\AppData\Local\Temp\ipykernel_20040\3540862808.py:5: DeprecationWarning: Redis.hmset() is deprecated. Use Redis.hset() instead.
  r.hmset(temp_download_table,table)


In [ ]:
while r.hlen(temp_download_table)>0:
    name = r.get('curr')
    #可从中断处返回

    if name == None:
        user= helper.get_user(r,temp_download_table)
        name = user[0]
        r.set('curr',name)
        urls = re.sub('[\[\] \']','',user[1]).split(',')
        for url in urls:
            r.hset(name,url,0)

    while r.hlen(name) > 0:
        url = helper.redis_hpop(r,name)  
        try :
            helper.down_load_video_from_user_homepage(redis.ConnectionPool(host='127.0.0.1',port=6379,db =0,password='123456',decode_responses=True)  ,name,url)
            r.hdel(name,url)
            time.sleep(random.uniform(0.1,1.1))
        except Exception:
            continue
        
    r.delete('curr',name)
    helper.delete_user(r,temp_download_table,name)

In [ ]:
helper.restore_table(pool,origin_table,temp_download_table,)
helper.check_empty_dir_and_update_redis(pool,"./video",temp_download_table,tolerence=0.90)